In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout

from sklearn.preprocessing import StandardScaler
from keras import metrics

Using TensorFlow backend.


In [2]:
plt.style.use('ggplot')
plt.rc('patch', force_edgecolor=True,edgecolor='black')
plt.rc('hist', bins='auto')

In [17]:
train = pd.read_csv('wrangled_data/training_post_wrangle.csv')

test_final = test_full.loc[:,['ID', 'shop_id', 'item_id']]

In [18]:
train

,shop_id,item_id,year,month,item_cnt_month,date_block_num,mean_item,mode_item,item_name,item_category_id,item_category_name,shop_name
0,0,30,2013,2,31.0,1.0,0.911765,0.0,30,40,40,0
1,0,30,2013,1,0.0,0.0,0.911765,0.0,30,40,40,0
2,0,30,2013,6,0.0,5.0,0.911765,0.0,30,40,40,0
3,0,30,2013,3,0.0,2.0,0.911765,0.0,30,40,40,0
4,0,30,2013,4,0.0,3.0,0.911765,0.0,30,40,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14411677,59,22167,2015,5,0.0,28.0,0.117647,0.0,22165,49,49,59
14411678,59,22167,2015,10,0.0,33.0,0.117647,0.0,22165,49,49,59
14411679,59,22167,2015,7,0.0,30.0,0.117647,0.0,22165,49,49,59
14411680,59,22167,2015,9,0.0,32.0,0.117647,0.0,22165,49,49,59


In [19]:
train = train[['shop_id' ,'item_id', 'item_cnt_month', 'date_block_num']]

In [23]:
train = train.set_index(['shop_id', 'item_id', 'date_block_num']).unstack()
train.shape

item_cnt_month                                                \
date_block_num            0.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0    
shop_id item_id                                                                
0       30                 0.0  31.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31                 0.0  11.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32                 6.0  10.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33                 3.0   3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        35                 1.0  14.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...   ...  ...  ...  ...  ...  ...  ...  ...  ...   
59      22154              1.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22155              0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
        22162              0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22164              0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22167              0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   

                 ...                                                    
date_block_num   ... 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0  
shop_id item_id  ...                                                    
0       30       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        31       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        32       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        33       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        35       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
59      22154    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        22155    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        22162    ...  0.0  9.0  4.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  
        22164    ...  0.0  2.0  1.0  2.0  0.0  0.0  1.0  0.0  0.0  0.0  
        22167    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[423873 rows x 34 columns]

In [26]:
train.shape

(423873, 34)

In [25]:
train.loc[slice(0), :]

item_cnt_month                                                \
date_block_num            0.0   1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0    
shop_id item_id                                                                
0       30                 0.0  31.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31                 0.0  11.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32                 6.0  10.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33                 3.0   3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        35                 1.0  14.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...   ...  ...  ...  ...  ...  ...  ...  ...  ...   
        22139              0.0   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22140              1.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22151             20.0   8.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22154              0.0   2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        22160              1.0   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                 ...                                                    
date_block_num   ... 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0  
shop_id item_id  ...                                                    
0       30       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        31       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        32       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        33       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        35       ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
        22139    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        22140    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        22151    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        22154    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        22160    ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[3600 rows x 34 columns]

In [27]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)

In [28]:
X_train = train_scaled[:,:-1]
X_predict = train_scaled[:,1:]
y_train = train_scaled[:,-1]

print(X_train.shape, y_train.shape, X_predict.shape)

(423873, 33) (423873,) (423873, 33)


In [29]:
X_train = X_train.reshape((423873, 33, -1))
X_predict = X_predict.reshape((423873, 33, -1))
y_train = y_train.reshape((423873, -1))

print(X_train.shape, y_train.shape, X_predict.shape)

(423873, 33, 1) (423873, 1) (423873, 33, 1)


In [38]:
model_lstm = tf.keras.Sequential()
model_lstm.add(tf.keras.layers.LSTM(1, activation='relu', input_shape = (X_train.shape[1], X_train.shape[2]), return_sequences=True))
model_lstm.add(tf.keras.layers.LeakyReLU(alpha=0.3))
model_lstm.add(tf.keras.layers.Dropout(0.3))
model_lstm.add(tf.keras.layers.Dense(3600))

model_lstm.compile(loss = 'mse',
                   optimizer = 'Adam',  
                   metrics = ['accuracy']) 
model_lstm.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 33, 1)             12        
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 33, 1)             0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 33, 1)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 33, 3600)          7200      
Total params: 7,212
Trainable params: 7,212
Non-trainable params: 0
_________________________________________________________________


In [39]:
history_lstm = model_lstm.fit(X_train, y_train, batch_size = 3600, epochs = 5)

ValueError: A target array with shape (423873, 1) was passed for an output of shape (None, 33, 3600) while using as loss `mean_squared_error`. This loss expects targets to have the same shape as the output.